# use trained model to another dataset

# 1.remove doggy image

In [ ]:
import tensorflow as tf
import os

In [ ]:
import cv2
import imghdr

In [ ]:
#２クラスのフォルダが格納されているディレクトリ名を指定
data_dir2 = 'snake-dataset-indian' 

In [ ]:
#受けつける拡張子の指定
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
#data_dirの中身のディレクトリを出力する。
#毒あり毒なしそれぞれのディレクトリの名前が出力されれば成功、
for image_class in os.listdir(data_dir2): 
    print(image_class)

In [ ]:
#data_dirの中に.DS_Storeがあった場合取り除くコード。

for image_class in os.listdir(data_dir2):
    if image_class == '.DS_Store':
        file_path = os.path.join(data_dir2, image_class)
        os.remove(file_path)
    else:
        print(image_class)

In [ ]:
#.DS_Storeが消えているのを確認
for image_class in os.listdir(data_dir2): 
    print(image_class)

In [ ]:
#対象としていない拡張子と壊れている画像ファイルをremoveする。
for image_class in os.listdir(data_dir2): 
    for image in os.listdir(os.path.join(data_dir2, image_class)):
        image_path = os.path.join(data_dir2, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            os.remove(image_path)

0と1がどのクラスに対応しているのかを出力する。

In [ ]:
class_mapping = {}   # クラス名とラベルの対応付けを格納する辞書

for label, image_class in enumerate(os.listdir(data_dir2)):
    if image_class == '.DS_Store':
        file_path = os.path.join(data_dir2, image_class)
        os.remove(file_path)
    else:
        class_name = image_class.split('_')[0]
        print(f"{class_name}: {label}")
        class_mapping[label] = class_name   

In [ ]:
print(class_mapping)

# 2.load data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#指定されたディレクトリから画像データセットを作成
data= tf.keras.utils.image_dataset_from_directory('snake-dataset-indian')

In [ ]:
#データセットの要素をnumpy配列として返すiteratorを作成。
data_iterator = data.as_numpy_iterator()

In [ ]:
#イテレータから次の要素を取得する
batch = data_iterator.next()

In [ ]:
#バッチの上から４つ選び、プロットする。

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 3.scale data

In [ ]:
#x:image
#y:target
#scale data to 0 through 1

data = data.map(lambda x,y: (x/255, y))

In [ ]:
#データセットの要素をnumpy配列として返すiteratorを作成。
#イテレータから次のバッチを取得する

data.as_numpy_iterator().next()

# 4.split data

In [ ]:
#バッチ数をプリント
len(data)

In [ ]:
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = len(data) - train_size - val_size

以下の3変数の合計がlen(data)分になるようにする。

In [ ]:
len(train_size )

In [ ]:
len(val_size)

In [ ]:
len(test_size)

In [ ]:
#take():how much we take from dataset
#skip():usually,skip take() area 

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 5.use model

In [ ]:
from keras.models import load_model

In [ ]:
#保存したモデルをloadする
#使用したいモデルの名前に置き換える。

new_model = load_model('models/INSERTDESIGNATEDMODELNAME.h5')

In [ ]:
from keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
#精度、再現率、および二値分類精度を計算し、
#真のラベルyと予測されたラベルyhatを比べる

for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = new_model.predict(X) 
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
#それぞれの出力値は1に近いほどよい

print(pre.result(), re.result(), acc.result())

# bonus: pass single image that our model has never seen before  to verify

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#モデルが一度も見たことがない画像1枚を渡す
new_image="" 

In [ ]:
#モデルが一度も見たことがない画像を出力

img = cv2.imread(new_image)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
#画像の前処理を行う

resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
#前処理をした画像をモデルに入力し、予測を行う。

yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
#予測値が 0.5 より大きい場合は、「予測されたクラスは非毒性」と表示される。
#それ以外の場合は、「予測されたクラスは有毒」と表示。

if yhat > 0.5: 
    print(f'Predicted class is NonPoison')
else:
    print(f'Predicted class is Poison')